In [107]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import gc
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

In [108]:
'''
오류가 있다.
28일 뒤를 예측하도록 해야한다 
shift를 시켜야한다.
'''

'\n오류가 있다.\n28일 뒤를 예측하도록 해야한다 \nshift를 시켜야한다.\n'

In [109]:
df_train = pd.read_csv('m5-forecasting-accuracy/sales_train_validation.csv')
calendar = pd.read_csv('m5-forecasting-accuracy/calendar.csv')
price = pd.read_csv('m5-forecasting-accuracy/sell_prices.csv')
# df_test = pd.read_csv('m5-forecasting-accuracy/sample_submission.csv')

In [110]:
# # startpoints 찾아서 이전 데이터 지우기
# startpoints = np.zeros(df_train.shape[0])
# for idx in tqdm(range(df_train.shape[0])):
#     startpoints[idx]= np.where(df_train.iloc[idx,6:].values>0)[0].min().astype(int)

In [111]:
# start_dict = dict(zip(df_train['id'], startpoints))

In [112]:
# df_train['key'] = df_train['id'].apply(lambda x : x[:-11])


In [113]:
# test 데이터 변환
# df_test = pd.read_csv('m5-forecasting-accuracy/sample_submission.csv')

# lst = list(calendar[1941:]['d'])
# for i in range(1,29):
#     df_test = df_test.rename(columns = {'F%s'%i:lst[i-1]})

# # Unpivot
# df_test = pd.melt(df_test, id_vars=df_test.columns[:1], value_vars=df_test.columns[1:],
#        var_name = 'day', value_name = 'volume')

# df_test['key'] = df_test['id'].apply(lambda x : x[:-11])
# # df_train에 있는 기본 정보 붙이기
# df_test = pd.merge(df_test,df_train.loc[:,['key','item_id','dept_id','cat_id','store_id','state_id']],
#          on =['key'], how='left')

# df_train.drop(['key'],axis =1, inplace =True)
# df_test.drop(['key'], axis = 1, inplace =True)
    

In [114]:
# test_id = df_test['id']


### 전략
1. Unpivot만 해서 LGBM 하기 
2. 날짜 인코딩만해서 LGBM 사용하기
3. Data preprocessing 어떻게 할 것인지?
4. 최근 며칠간의 평균 판매액
5. rolling 추가
6. validation set을 6월 데이터 를 예측하는 것으로 해볼까?
  - 2015년 6월 데이터 예측 ? 

In [115]:
df_train.head()
print(df_train.shape)


(30490, 1919)


In [116]:
# F_1~28 만들기

tr_last = 1913
# tst = pd.read_csv('m5-forecasting-accuracy/sales_train_validation.csv')

# 1914~1941 까지 
for i in range(tr_last+1, tr_last+1+28):
    df_train['d_%s'%i] = 0


In [117]:
df_train.dtypes

id          object
item_id     object
dept_id     object
cat_id      object
store_id    object
             ...  
d_1937       int64
d_1938       int64
d_1939       int64
d_1940       int64
d_1941       int64
Length: 1947, dtype: object

In [118]:
# numcols = [f"d_{day}" for day in range(1, 1913+1)]

# catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
# dtype = {numcol:"float32" for numcol in numcols} 
# dtype.update({col:"category" for col in catcols if col != "id"})
# df_train = pd.read_csv('m5-forecasting-accuracy/sales_train_validation.csv',dtype = dtype,
#                       usecols = catcols + numcols)
# print(df_train.shape)
# for col in catcols : # category 컬럼 encoding
#     if col != "id":
#         df_train[col] = df_train[col].cat.codes.astype("int16")
#         df_train[col] -= df_train[col].min()

In [119]:
# # Unpivot
df_train = pd.melt(df_train, id_vars=df_train.columns[:6], value_vars=df_train.columns[6:],
       var_name = 'day', value_name = 'volume')
# df_train = pd.melt(df_train, id_vars = catcols,
#             value_vars= [col for col in df_train.columns if col.startswith("d_")],
#             var_name ="day",
#             value_name = "volume")
print(df_train.shape)

    

(59181090, 8)


In [120]:
# print(numcols[:10])
# catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
# dtype = {numcol:"float32" for numcol in numcols} 
# dtype.update({col:"category" for col in catcols if col != "id"})
# # 데이터를 다 불러오지 않고 필요한 데이터만 불러왔다.
# # startdate도 정해뒀다.
# dt = pd.read_csv("m5-forecasting-accuracy/sales_train_validation.csv",
#                  usecols = catcols + numcols,dtype = dtype)
# print(dt.shape)
# for col in catcols : # category 컬럼 encoding
#     if col != "id":
#         dt[col] = dt[col].cat.codes.astype("int16")
#         dt[col] -= dt[col].min()

            
# dt = pd.melt(dt, id_vars = catcols,
#             value_vars= [col for col in dt.columns if col.startswith("d_")],
#             var_name ="d",
#             value_name = "sales")
# print(dt.shape)

In [121]:
# calendar 추가하기
df_train = pd.merge(df_train, calendar, left_on = 'day', right_on ='d')
# df_test = pd.merge(df_test, calendar, left_on = 'day', right_on ='d')

# df_train = df_train.fillna("NaN")
# df_test = df_test.fillna("NaN")

In [122]:
print(df_train.shape)


(59181090, 22)


In [123]:
df_train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,volume,date,wm_yr_wk,...,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0


In [124]:
drop_cols = []
cat_cols = []
drop_cols += ['date','d','id']



In [125]:
# snap 합치기
snap = np.zeros(df_train.shape[0])
snap[df_train[(df_train['state_id']=='CA')&(df_train['snap_CA']==1)].index] +=1
snap[df_train[(df_train['state_id']=='TX')&(df_train['snap_TX']==1)].index] +=1
snap[df_train[(df_train['state_id']=='WI')&(df_train['snap_WI']==1)].index] +=1
df_train['snap'] = snap

# test
# snap = np.zeros(df_test.shape[0])
# snap[df_test[(df_test['state_id']=='CA')&(df_test['snap_CA']==1)].index] +=1
# snap[df_test[(df_test['state_id']=='TX')&(df_test['snap_TX']==1)].index] +=1
# snap[df_test[(df_test['state_id']=='WI')&(df_test['snap_WI']==1)].index] +=1
# df_test['snap'] = snap

drop_cols += ['snap_CA','snap_TX','snap_WI']



In [126]:

cat_cols += [ 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id',
#               'wday', 'month', 'year', # 이게 크리티컬?
            'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap']

In [127]:
# Sell price
df_train.head()
df_train = pd.merge(df_train, price)
# df_test = pd.merge(df_test,price)
# price
print(df_train.shape)


(46881677, 24)


In [128]:
# df_train['d'] >='d_'+df_train['startpoint']

In [129]:
# # Start point 찾기 ::  40% 데이터를 날릴  수 있다.  -- price가 책정 된 날이 시작 점???

# df_train['startpoint'] = df_train['id'].map(start_dict).astype(int).astype(str)
# df_train['startpoints'] = df_train['d'] >='d_'+df_train['startpoint']
# print(df_train['startpoints'].value_counts())
# print(df_train.shape)
# # df_train = df_train[df_train['startpoints']]
# # print(df_train.shape)
# # df_train.drop(['startpoint','startpoints'],axis =1, inplace= True)

In [130]:
%%time

# 왜 Shift를 해야할까?
# Shift 28을 하지 않으면 예측값이 뒤로 가면갈 수록  F1->F28 예측 할 수 있는 변수가 줄어든게 된다.
# 28일은 한달을 의미한다. 최근 한달간의 경향성을 보는 것으로 보면 되겠다.
# 28일을 56일로 늘리면 안되나? - 최근 한달간의 경향성이 반영이 안되는 것이낙?
# https://www.kaggle.com/kneroma/m5-first-public-notebook-under-0-50

df_train['volume_7'] = df_train[['id','volume']].groupby("id")['volume'].shift(7)
df_train['volume_28'] = df_train[['id','volume']].groupby("id")['volume'].shift(28)

print("mean")

df_train['rmean_7_7'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).mean())
df_train['rmean_7_28'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).mean())
# df_train['rmean_7_50'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).mean())

df_train['rmean_28_7'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).mean())
df_train['rmean_28_28'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).mean())
# df_train['rmean_28_50'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).mean())


# df_train.dropna(inplace=True)


mean
Wall time: 2min 19s


In [131]:
df_train['date'] =  pd.to_datetime(df_train["date"])

In [132]:
df_train['week'] = getattr(df_train["date"].dt, "weekofyear").astype("int16")
df_train['quarter'] = getattr(df_train["date"].dt,"quarter").astype("int16")
df_train['mday'] = getattr(df_train["date"].dt, "day").astype("int16")




In [133]:
cols =['event_name_1','event_type_1','event_name_2','event_type_2']
df_train[cols]= df_train[cols].fillna('NaN')
print(df_train.shape)
# df_train.dropna(inplace =True)
# df_train.isnull().sum()
# print(df_train.shape)

(46881677, 33)


In [134]:
df_train.isnull().sum()

id                    0
item_id               0
dept_id               0
cat_id                0
store_id              0
state_id              0
day                   0
volume                0
date                  0
wm_yr_wk              0
weekday               0
wday                  0
month                 0
year                  0
d                     0
event_name_1          0
event_type_1          0
event_name_2          0
event_type_2          0
snap_CA               0
snap_TX               0
snap_WI               0
snap                  0
sell_price            0
volume_7         213430
volume_28        853720
rmean_7_7        396370
rmean_7_28      1036660
rmean_28_7      1036660
rmean_28_28     1676950
week                  0
quarter               0
mday                  0
dtype: int64

In [135]:
df_train.dropna(inplace =True)

In [136]:
df_train.shape

(45204727, 33)

In [137]:
#  "wm_yr_wk", "weekday" 추가 삭제
#     'snap_CA', 'snap_TX', 'snap_WI' 삭제 안함
    

In [138]:
drop_cols += ["wm_yr_wk", "weekday"]  ## 이게 문제?
drop_cols

['date', 'd', 'id', 'snap_CA', 'snap_TX', 'snap_WI', 'wm_yr_wk', 'weekday']

In [139]:
# %%time

# df_train['train'] = True
# # df_test['train']  = False
# # full_df = pd.concat([df_train,df_test])

# full_df['volume_shift_28'] =full_df[['id','volume']].groupby("id")['volume'].shift(28)

# print("mean")
# full_df['rmean_7'] = full_df[['id','volume_shift_28']].groupby("id")['volume_shift_28'].transform(lambda x: x.rolling(7 ,min_periods=1).mean())
# full_df['rmean_28'] = full_df[['id','volume_shift_28']].groupby("id")['volume_shift_28'].transform(lambda x: x.rolling(28 ,min_periods=1).mean())
# full_df['rmean_50'] = full_df[['id','volume_shift_28']].groupby("id")['volume_shift_28'].transform(lambda x: x.rolling(50 ,min_periods=1).mean())

# # print("std")
# # full_df['rstd_7'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(7 ,min_periods=1).std())
# # full_df['rstd_28'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(28 ,min_periods=1).std())
# # full_df['rstd_50'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(50 ,min_periods=1).std())

# # print("max")
# # full_df['rmax_7'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(7 ,min_periods=1).max())
# # full_df['rmax_28'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(28 ,min_periods=1).max())
# # full_df['rmax_50'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(50 ,min_periods=1).max())

# # print("min")
# # full_df['rmin_7'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(7 ,min_periods=1).min())
# # full_df['rmin_28'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(28 ,min_periods=1).min())
# # full_df['rmin_50'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(50 ,min_periods=1).min())

# # print("count")
# # full_df['rcnt_7'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(7).count() if x>0).fllna(0)
# # full_df['rcnt_28'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(28).count() if x>0).fllna(0)
# # full_df['rcnt_50'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(28).count() if x>0).fllna(0)

# full_df = full_df.fillna(0)

# df_train = full_df[full_df['train'] ==True].drop(['train'], axis =1)
# df_test = full_df[full_df['train'] ==False].drop(['train'], axis =1)

# del full_df
# gc.collect()


In [140]:
df_train.head()
tr_last = 1913
testday = ['d_%s'% x for x in range(tr_last+1, tr_last+1+28)]
train_id = df_train['id']
df_test_id = df_train[df_train['day'].isin(testday)]['id']

In [141]:
df_train= df_train.drop(drop_cols,axis =1 )
# df_test =df_test.drop(drop_cols,axis =1)

In [142]:
df_train.head()

,item_id,dept_id,cat_id,store_id,state_id,day,volume,wday,month,year,...,sell_price,volume_7,volume_28,rmean_7_7,rmean_7_28,rmean_28_7,rmean_28_28,week,quarter,mday
617364,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_56,0,7,3,2011,...,0.42,0.0,0.0,2.000000,1.642857,4.000000,4.535714,12,1,25
617371,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_56,0,7,3,2011,...,1.77,0.0,0.0,2.000000,2.142857,2.000000,2.178571,12,1,25
617378,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,d_56,0,7,3,2011,...,3.17,0.0,0.0,0.000000,0.035714,0.142857,0.214286,12,1,25
617385,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,d_56,1,7,3,2011,...,6.27,1.0,1.0,0.428571,0.535714,0.857143,0.678571,12,1,25
617392,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,d_56,1,7,3,2011,...,0.72,2.0,1.0,5.428571,6.142857,8.000000,4.321429,12,1,25


In [143]:
# Encoding
for col in tqdm(cat_cols) :  # encoding -1이 문제?
    le = LabelEncoder()
#     le.fit()
#     le.fit(pd.concat([df_train[col],df_test[col]]))
    df_train[col] = le.fit_transform(df_train[col]).astype(np.int8)
#     df_test[col] = le.transform(df_test[col]).astype(np.int8)
    gc.collect()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:05<00:00,  6.57s/it]


In [144]:
max_lag = 56

In [145]:
# Test 분리하기
testday = ['d_%s'% x for x in range(tr_last+1, tr_last+1+28)]
df_test = df_train.copy()#df_train[df_train['day'].isin(testday)]
df_train = df_train[~df_train['day'].isin(testday)]

# testday
df_train[df_train['day']== 'd_1914'].head()
# tst[tst['day']== 'd_1914']

,item_id,dept_id,cat_id,store_id,state_id,day,volume,wday,month,year,...,sell_price,volume_7,volume_28,rmean_7_7,rmean_7_28,rmean_28_7,rmean_28_28,week,quarter,mday


In [41]:
df_test.head()

,item_id,dept_id,cat_id,store_id,state_id,day,volume,wday,month,year,...,sell_price,volume_7,volume_28,rmean_7_7,rmean_7_28,rmean_28_7,rmean_28_28,week,quarter,mday
45966979,-99,3,1,0,0,d_1914,0,3,4,2016,...,8.38,1.0,1.0,0.857143,0.964286,1.142857,1.035714,17,2,25
45966980,-99,3,1,0,0,d_1915,0,4,4,2016,...,8.38,1.0,0.0,1.000000,0.928571,0.857143,0.964286,17,2,26
45966981,-99,3,1,0,0,d_1916,0,5,4,2016,...,8.38,1.0,0.0,1.142857,0.892857,0.571429,0.964286,17,2,27
45966982,-99,3,1,0,0,d_1917,0,6,4,2016,...,8.38,3.0,0.0,1.428571,1.000000,0.571429,0.928571,17,2,28
45966983,-99,3,1,0,0,d_1918,0,7,4,2016,...,8.38,0.0,0.0,1.285714,0.964286,0.428571,0.892857,17,2,29


In [42]:
# # Train_test_split  -- 가장 마지막 28일을 예측하는 것으로 짜보자
# # lst = [0 for x in range(df_train.shape[0])]

# x_train = df_train[~df_train['day'].isin(df_train['day'].unique()[-100:])]
# x_valid = df_train[df_train['day'].isin(df_train['day'].unique()[-100:])]

# y_train, y_valid = x_train['volume'], x_valid['volume']

# x_train = x_train.drop(['day','volume'], axis =1)
# x_valid = x_valid.drop(['day','volume'], axis =1)

In [43]:
%%time

from sklearn.model_selection import train_test_split

x_train , x_valid = train_test_split(df_train, test_size =0.15, random_state = 99)
y_train, y_valid = x_train['volume'], x_valid['volume']

x_train = x_train.drop(['day','volume'], axis =1)
x_valid = x_valid.drop(['day','volume'], axis =1)

Wall time: 54.2 s


In [44]:
x_valid.shape[0]/x_train.shape[0]

0.17647061788239832

In [45]:
from sklearn.metrics import mean_squared_log_error
seed = 99

In [46]:
# # lightbgm
# params = {
#     'boosting_type': 'gbdt',
#     'objective': 'regression',
#     'metric': {'rmse'},#{'l2'},
#     #'subsample': 0.2,
#     'learning_rate': 0.1,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.9,
# #     'num_leaves': 128,
#     'alpha': 0.1,
#     'lambda': 0.1,
#     'n_jobs' :10 
# }

params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 1200,
    'num_leaves': 128,
    "min_data_in_leaf": 100,
}

In [47]:
%%time
# Modeling
lgb_train = lgb.Dataset(x_train, y_train,categorical_feature=cat_cols)
lgb_eval = lgb.Dataset(x_valid, y_valid,categorical_feature=cat_cols)
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1000, #300,
                valid_sets=(lgb_train, lgb_eval),
#                     feval=rmsle,
#                 early_stopping_rounds= 50,#100,
                verbose_eval=50) #100)

C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\lightgbm\basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[50]	training's rmse: 2.58517	valid_1's rmse: 2.61566
[100]	training's rmse: 2.49621	valid_1's rmse: 2.53045
[150]	training's rmse: 2.47427	valid_1's rmse: 2.51027
[200]	training's rmse: 2.45713	valid_1's rmse: 2.49499
[250]	training's rmse: 2.44043	valid_1's rmse: 2.48062
[300]	training's rmse: 2.42782	valid_1's rmse: 2.47053
[350]	training's rmse: 2.41513	valid_1's rmse: 2.46159
[400]	training's rmse: 2.40332	valid_1's rmse: 2.45356
[450]	training's rmse: 2.39264	valid_1's rmse: 2.44612
[500]	training's rmse: 2.38333	valid_1's rmse: 2.44009
[550]	training's rmse: 2.37442	valid_1's rmse: 2.43536
[600]	training's rmse: 2.36698	valid_1's rmse: 2.43103
[650]	training's rmse: 2.3592	valid_1's rmse: 2.42714
[700]	training's rmse: 2.35329	valid_1's rmse: 2.42432
[750]	training's rmse: 2.34726	valid_1's rmse: 2.42095
[800]	training's rmse: 2.34159	valid_1's rmse: 2.41868
[850]	training's rmse: 2.33639	valid_1's rmse: 2.41625
[900]	training's rmse: 2.33165	valid_1's rmse: 2.4141
[950]	trainin

In [48]:
gbm.save_model("검증_model.lgb")

In [180]:
%%time
df_test['id'] =train_id
max_lag  = 56
trn_lst = 1913
for tdelta in range(1, 29):
    f_day = trn_lst+tdelta
    print(f_day)
    days = [f"d_{i}" for i in range(trn_lst-max_lag+tdelta,f_day+1)]
    tst = df_test[df_test['day'].isin(days)]

    print("rolling")
    tst['volume_7'] = tst[['id','volume']].groupby("id")['volume'].shift(7)
    tst['volume_28'] = tst[['id','volume']].groupby("id")['volume'].shift(28)
    
    print("mean")
    tst['rmean_7_7'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).mean())
    tst['rmean_7_28'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).mean())

    tst['rmean_28_7'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).mean())
    tst['rmean_28_28'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).mean())

    tst = tst[tst['day'] == "d_%s"%(f_day)]
    t_id,t_volume,t_day = tst['id'],tst['volume'],tst['day']
    tst = tst.drop(['id','volume','day'], axis =1)
    df_test.loc[df_test.day=="d_%s"%(f_day),'volume'] = 1.028*gbm.predict(tst)
    

1914
rolling


C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


mean


C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

1915
rolling
mean
1916
rolling
mean
1917
rolling
mean
1918
rolling
mean
1919
rolling
mean
1920
rolling
mean
1921
rolling
mean
1922
rolling
mean
1923
rolling
mean
1924
rolling
mean
1925
rolling
mean
1926
rolling
mean
1927
rolling
mean
1928
rolling
mean
1929
rolling
mean
1930
rolling
mean
1931
rolling
mean
1932
rolling
mean
1933
rolling
mean
1934
rolling
mean
1935
rolling
mean
1936
rolling
mean
1937
rolling
mean
1938
rolling
mean
1939
rolling
mean
1940
rolling
mean
1941
rolling
mean
Wall time: 22min 33s


In [181]:

cols = [f"d_{i}" for i in range(1914,1942)]

sub = df_test[df_test['day'].isin(cols)].loc[:,['id','volume']]
sub['F']= [f"F{rank}" for rank in sub.groupby("id")["id"].cumcount()+1]
sub = sub.set_index(["id", "F" ]).unstack()["volume"].reset_index()
sub.sort_values("id", inplace = True)
sub.reset_index(drop=True, inplace = True)                                                   
sub =sub[['id']+["F%s"% x for x in range(1,29)]]


sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submission_20200430_4.csv",index=False)

In [182]:
sub

F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.981515,0.922311,0.896392,0.866220,1.223651,1.265083,1.236177,1.044582,1.026049,...,1.134804,1.333534,1.244420,1.040998,0.950892,0.921802,0.944397,1.097852,1.246530,1.268082
1,FOODS_1_001_CA_2_validation,1.289609,1.389979,1.248224,1.531910,1.794326,1.889266,1.706961,1.030615,1.061157,...,1.381096,1.771509,1.811014,1.313962,1.238412,1.222575,1.314619,1.517834,1.873056,1.617837
2,FOODS_1_001_CA_3_validation,1.118587,1.081466,1.039929,1.019106,1.145837,1.266579,1.056370,0.978135,1.040048,...,1.134021,1.410956,1.504408,1.123811,1.062363,1.043118,1.050028,1.157301,1.308888,1.225178
3,FOODS_1_001_CA_4_validation,0.399603,0.353737,0.351917,0.353929,0.440999,0.461881,0.519347,0.383425,0.419510,...,0.407992,0.425698,0.464631,0.376035,0.358540,0.380014,0.381644,0.438157,0.461234,0.488663
4,FOODS_1_001_TX_1_validation,0.167414,0.166010,0.153484,0.154642,0.145016,0.142718,0.164538,0.568960,0.558801,...,0.406596,0.374682,0.376486,0.262124,0.253114,0.244448,0.246525,0.280588,0.306032,0.334676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516_TX_2_evaluation,0.293089,0.261911,0.300074,0.292941,0.330569,0.387549,0.411654,0.289743,0.304022,...,0.313562,0.415208,0.444078,0.293818,0.278588,0.291964,0.286129,0.310466,0.405064,0.384740
30486,HOUSEHOLD_2_516_TX_3_evaluation,0.179081,0.157190,0.161648,0.152205,0.170801,0.177646,0.183672,0.152773,0.143976,...,0.181387,0.182285,0.185505,0.152849,0.151746,0.150905,0.152316,0.167070,0.191568,0.213417
30487,HOUSEHOLD_2_516_WI_1_evaluation,0.098089,0.096666,0.098329,0.100661,0.094170,0.110451,0.097827,0.125112,0.125493,...,0.125522,0.147102,0.144949,0.106606,0.106047,0.107925,0.109844,0.132252,0.156633,0.148824
30488,HOUSEHOLD_2_516_WI_2_evaluation,0.059283,0.057786,0.057974,0.113240,0.111405,0.118056,0.117481,0.125470,0.127363,...,0.149960,0.162013,0.151819,0.136363,0.131802,0.134746,0.105300,0.115359,0.124854,0.132803


In [186]:
df_test['volume']

617364      0.000000
617371      0.000000
617378      0.000000
617385      1.000000
617392      1.000000
              ...   
46881672    0.994041
46881673    1.212759
46881674    1.220009
46881675    1.765072
46881676    1.810848
Name: volume, Length: 45204727, dtype: float64

In [146]:
%%time



df_test['id'] = df_test_id
df_test['volume'] = predict
df_test['day'] =df_day
tst =df_test.loc[:,['id','day','volume']]
tst['F']= [f"F{rank}" for rank in tst.groupby("id")["id"].cumcount()+1]
tst = tst.set_index(["id", "F" ]).unstack()["volume"].reset_index()
tst.sort_values("id", inplace = True)
tst.reset_index(drop=True, inplace = True)                                                   
tst =tst[['id']+["F%s"% x for x in range(1,29)]]

sub2 = tst.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([tst, sub2], axis=0, sort=False)                     

sub.fillna(0, inplace = True)
sub


cols = [f"d_{i}" for i in range(1914,1942)]
# predict
# df_test
for i in range(0,28):    
    tst =  df_test[df_test['day']==cols[0]]
    t_id,t_volume,t_day = tst['id'],tst['volume'],tst['day']
    tst = tst.drop(['id','volume','day'], axis =1)
    df_test.loc[df_test.day==cols[0],'volume'] = 1.028*gbm.predict(tst)
    
    

1

In [ ]:
df_train['volume_7'] = df_train[['id','volume']].groupby("id")['volume'].shift(7)
df_train['volume_28'] = df_train[['id','volume']].groupby("id")['volume'].shift(28)

print("mean")

df_train['rmean_7_7'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).mean())
df_train['rmean_7_28'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).mean())
# df_train['rmean_7_50'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).mean())

df_train['rmean_28_7'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).mean())
df_train['rmean_28_28'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).mean())


In [68]:
df_test = df_test.drop(['volume'],axis =1)

In [75]:
%%time
df_day = df_test['day']
df_test = df_test.drop(['day'],axis =1 )

predict = gbm.predict(df_test)
# predict = np.around(predict)

Wall time: 9.88 s


In [76]:
predict

array([0.85868253, 0.77328878, 0.77238252, ..., 0.24929779, 0.63705935,
       0.62918988])

In [51]:
# predict = np.around(predict)

In [77]:
df_test['id'] = df_test_id
df_test['volume'] = predict
df_test['day'] =df_day
tst =df_test.loc[:,['id','day','volume']]
tst['F']= [f"F{rank}" for rank in tst.groupby("id")["id"].cumcount()+1]
tst = tst.set_index(["id", "F" ]).unstack()["volume"].reset_index()
tst.sort_values("id", inplace = True)
tst.reset_index(drop=True, inplace = True)                                                   
tst =tst[['id']+["F%s"% x for x in range(1,29)]]

sub2 = tst.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([tst, sub2], axis=0, sort=False)                     

sub.fillna(0, inplace = True)
sub

F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.954781,0.897190,0.871976,0.842626,1.190322,1.230626,1.202507,0.941322,0.891179,...,0.352754,0.398432,0.403311,0.271497,0.224389,0.208768,0.205712,0.216577,0.234634,0.232262
1,FOODS_1_001_CA_2_validation,1.254484,1.352120,1.214225,1.490185,1.745453,1.837808,1.660468,0.933029,0.897058,...,0.665104,0.849939,0.575913,0.392221,0.337962,0.299269,0.277079,0.296570,0.372924,0.251114
2,FOODS_1_001_CA_3_validation,1.088120,1.052010,1.011604,0.991348,1.114627,1.232080,1.027597,0.829422,0.847095,...,0.419339,0.600610,0.378497,0.254385,0.237811,0.228947,0.222177,0.227741,0.290733,0.176045
3,FOODS_1_001_CA_4_validation,0.388719,0.344102,0.342332,0.344289,0.428988,0.449301,0.505201,0.362176,0.368160,...,0.222524,0.227364,0.227153,0.193321,0.181271,0.173463,0.174524,0.184439,0.190753,0.177556
4,FOODS_1_001_TX_1_validation,0.162854,0.161488,0.149303,0.150430,0.141066,0.138830,0.160056,0.132287,0.125847,...,0.102841,0.095609,0.097695,0.089511,0.087857,0.090349,0.090177,0.098787,0.100004,0.100562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516_TX_2_evaluation,0.285106,0.254778,0.291901,0.284962,0.321566,0.376993,0.400441,0.274170,0.281959,...,0.212783,0.273935,0.231138,0.156891,0.150203,0.143082,0.141216,0.157919,0.162523,0.153692
30486,HOUSEHOLD_2_516_TX_3_evaluation,0.174203,0.152908,0.157245,0.148059,0.166149,0.172807,0.178670,0.138166,0.129425,...,0.140061,0.089830,0.090431,0.068073,0.065527,0.064866,0.065874,0.080741,0.097069,0.097742
30487,HOUSEHOLD_2_516_WI_1_evaluation,0.095418,0.094033,0.095651,0.097919,0.091605,0.107443,0.095163,0.073183,0.072505,...,0.083140,0.094553,0.089781,0.066210,0.064656,0.065760,0.067295,0.063733,0.082766,0.079662
30488,HOUSEHOLD_2_516_WI_2_evaluation,0.057668,0.056212,0.056395,0.110156,0.108371,0.114840,0.114281,0.099234,0.100444,...,0.082344,0.089919,0.084261,0.072032,0.068678,0.070170,0.064009,0.070403,0.077265,0.072303


In [78]:
submission = pd.read_csv('m5-forecasting-accuracy/sample_submission.csv')
submission.shape

(60980, 29)

In [79]:
submission =pd.merge(submission['id'],sub, on = 'id')
submission.to_csv("submission_20200430_문제점찾기3.csv",index=False)

In [80]:
# np.round(gbm.predict(df_test[df_test['day']=='d_1914'].drop(['id','day'],axis =1 )))
submission

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.858683,0.773289,0.772383,0.863592,0.964759,1.209123,1.356740,0.961900,0.908352,...,0.443610,0.568514,0.528916,0.367417,0.331517,0.320811,0.303106,0.323738,0.332273,0.327751
1,HOBBIES_1_002_CA_1_validation,0.199717,0.193180,0.248749,0.220073,0.248364,0.273053,0.263008,0.184352,0.180497,...,0.155090,0.182167,0.182142,0.135380,0.129058,0.125864,0.113465,0.122138,0.125795,0.115660
2,HOBBIES_1_003_CA_1_validation,0.603248,0.590754,0.592229,0.494465,0.620298,0.751751,0.728522,0.507541,0.469373,...,0.438566,0.536504,0.523692,0.370227,0.344734,0.321152,0.298255,0.305531,0.342081,0.300114
3,HOBBIES_1_004_CA_1_validation,1.734987,1.505665,1.415755,1.411780,1.991319,2.609851,2.509331,1.768090,1.645524,...,0.744480,1.241479,0.968174,0.577099,0.519647,0.543964,0.482330,0.540226,0.749562,0.537282
4,HOBBIES_1_005_CA_1_validation,0.950899,0.882103,0.932512,0.999228,1.224100,1.531993,1.791313,1.213099,1.118749,...,0.516487,0.680927,0.641379,0.460527,0.399023,0.384555,0.382088,0.422847,0.513496,0.499801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.236827,0.229253,0.203649,0.245926,0.231888,0.234793,0.287197,0.264115,0.267162,...,0.238062,0.290543,0.294901,0.204928,0.188782,0.183258,0.181237,0.172786,0.202431,0.183205
60976,FOODS_3_824_WI_3_evaluation,0.295459,0.250318,0.246477,0.246646,0.287265,0.370872,0.324295,0.303954,0.292711,...,0.189409,0.236496,0.220175,0.145022,0.142513,0.133687,0.131713,0.151641,0.161891,0.160117
60977,FOODS_3_825_WI_3_evaluation,0.879599,0.747451,0.783760,0.714545,0.787846,1.014324,0.980489,0.911389,0.855228,...,0.398339,0.504555,0.517682,0.292049,0.270063,0.239139,0.241858,0.234734,0.238778,0.244305
60978,FOODS_3_826_WI_3_evaluation,0.880078,0.795457,0.827549,0.714086,1.008884,1.153359,1.236396,1.053158,1.040677,...,0.366587,0.462824,0.446645,0.313692,0.288025,0.276131,0.275867,0.264282,0.282645,0.249298


In [86]:
cols = [f"d_{i}" for i in range(1914,1942)]
# predict
# df_test
for i in range(0,28):    
    tst =  df_test[df_test['day']==cols[0]]
    t_id,t_volume,t_day = tst['id'],tst['volume'],tst['day']
    tst = tst.drop(['id','volume','day'], axis =1)
    df_test.loc[df_test.day==cols[0],'volume'] = 1.028*gbm.predict(tst)
    
    

In [ ]:
df_test[['id','volume']].groupby("id")['volume'].shift(7)


In [99]:
# te_sub

df_train['volume_7'] = df_train[['id','volume']].groupby("id")['volume'].shift(7)
df_train['volume_28'] = df_train[['id','volume']].groupby("id")['volume'].shift(28)

print("mean")

df_train['rmean_7_7'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).mean())
df_train['rmean_7_28'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).mean())
# df_train['rmean_7_50'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).mean())

df_train['rmean_28_7'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).mean())
df_train['rmean_28_28'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).mean())


,id,volume,F
45966979,HOBBIES_1_001_CA_1_validation,0.882726,F1
45966980,HOBBIES_1_001_CA_1_validation,0.773289,F2
45966981,HOBBIES_1_001_CA_1_validation,0.772383,F3
45966982,HOBBIES_1_001_CA_1_validation,0.863592,F4
45966983,HOBBIES_1_001_CA_1_validation,0.964759,F5
...,...,...,...
46881672,FOODS_3_825_WI_3_validation,0.244305,F28
46881673,FOODS_3_826_WI_3_validation,0.282645,F27
46881674,FOODS_3_826_WI_3_validation,0.249298,F28
46881675,FOODS_3_827_WI_3_validation,0.637059,F27


In [101]:
cols = [f"F{i}" for i in range(1,29)]

te_sub = df_test.loc[:,["id","volume"]]
te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
te_sub = te_sub.set_index(["id", "F" ]).unstack()["volume"][cols].reset_index()
te_sub.fillna(0., inplace = True)
te_sub.sort_values("id", inplace = True)
te_sub.reset_index(drop=True, inplace = True)
    
sub2 = te_sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([te_sub, sub2], axis=0, sort=False)
sub.to_csv("submission_20200430_3.csv",index=False)

In [102]:
sub

F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.981515,0.897190,0.871976,0.842626,1.190322,1.230626,1.202507,0.941322,0.891179,...,0.352754,0.398432,0.403311,0.271497,0.224389,0.208768,0.205712,0.216577,0.234634,0.232262
1,FOODS_1_001_CA_2_validation,1.289609,1.352120,1.214225,1.490185,1.745453,1.837808,1.660468,0.933029,0.897058,...,0.665104,0.849939,0.575913,0.392221,0.337962,0.299269,0.277079,0.296570,0.372924,0.251114
2,FOODS_1_001_CA_3_validation,1.118587,1.052010,1.011604,0.991348,1.114627,1.232080,1.027597,0.829422,0.847095,...,0.419339,0.600610,0.378497,0.254385,0.237811,0.228947,0.222177,0.227741,0.290733,0.176045
3,FOODS_1_001_CA_4_validation,0.399603,0.344102,0.342332,0.344289,0.428988,0.449301,0.505201,0.362176,0.368160,...,0.222524,0.227364,0.227153,0.193321,0.181271,0.173463,0.174524,0.184439,0.190753,0.177556
4,FOODS_1_001_TX_1_validation,0.167414,0.161488,0.149303,0.150430,0.141066,0.138830,0.160056,0.132287,0.125847,...,0.102841,0.095609,0.097695,0.089511,0.087857,0.090349,0.090177,0.098787,0.100004,0.100562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516_TX_2_evaluation,0.293089,0.254778,0.291901,0.284962,0.321566,0.376993,0.400441,0.274170,0.281959,...,0.212783,0.273935,0.231138,0.156891,0.150203,0.143082,0.141216,0.157919,0.162523,0.153692
30486,HOUSEHOLD_2_516_TX_3_evaluation,0.179081,0.152908,0.157245,0.148059,0.166149,0.172807,0.178670,0.138166,0.129425,...,0.140061,0.089830,0.090431,0.068073,0.065527,0.064866,0.065874,0.080741,0.097069,0.097742
30487,HOUSEHOLD_2_516_WI_1_evaluation,0.098089,0.094033,0.095651,0.097919,0.091605,0.107443,0.095163,0.073183,0.072505,...,0.083140,0.094553,0.089781,0.066210,0.064656,0.065760,0.067295,0.063733,0.082766,0.079662
30488,HOUSEHOLD_2_516_WI_2_evaluation,0.059283,0.056212,0.056395,0.110156,0.108371,0.114840,0.114281,0.099234,0.100444,...,0.082344,0.089919,0.084261,0.072032,0.068678,0.070170,0.064009,0.070403,0.077265,0.072303


In [103]:
df_test.columns

Index(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'wday', 'month',
       'year', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap', 'sell_price', 'volume_7', 'volume_28', 'rmean_7_7',
       'rmean_7_28', 'rmean_28_7', 'rmean_28_28', 'week', 'quarter', 'mday',
       'id', 'volume', 'day'],
      dtype='object')

In [ ]:
# te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
# te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
# te_sub.fillna(0., inplace = True)
# te_sub.sort_values("id", inplace = True)
# te_sub.reset_index(drop=True, inplace = True)
# te_sub.to_csv(f"submission_{icount}.csv",index=False)

In [ ]:
sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submission.csv",index=False)

In [28]:
df_day = df_test['day']
df_test = df_test.drop(['day'],axis =1 )

In [29]:
%%time

predict = gbm.predict(df_test)
predict = np.around(predict)

Wall time: 19 s


In [30]:
submission = pd.read_csv('m5-forecasting-accuracy/sample_submission.csv')


In [33]:
df_test['id'] = test_id
df_test['predict']  = predict
df_test['day'] = df_day
df_test.head()

,item_id,dept_id,cat_id,store_id,state_id,volume,wm_yr_wk,weekday,wday,month,...,event_type_2,snap,sell_price,volume_shift_28,rmean_7,rmean_28,rmean_50,id,predict,day
0,-99,3,1,0,0,0,18,1,2,4,...,1,0,8.38,0.0,0.571429,0.464286,0.42,HOBBIES_1_001_CA_1_validation,1.0,d_1942
1,-99,3,1,0,0,0,18,1,2,4,...,1,0,8.38,0.0,0.000000,0.000000,0.00,HOBBIES_1_002_CA_1_validation,8.0,d_1942
2,-99,3,1,0,0,0,18,5,3,4,...,1,0,8.38,1.0,0.428571,0.464286,0.44,HOBBIES_1_003_CA_1_validation,1.0,d_1943
3,-99,3,1,0,0,0,18,5,3,4,...,1,0,8.38,0.0,0.000000,0.000000,0.00,HOBBIES_1_004_CA_1_validation,6.0,d_1943
4,-99,3,1,0,0,0,18,6,4,4,...,1,0,8.38,0.0,0.285714,0.464286,0.44,HOBBIES_1_005_CA_1_validation,1.0,d_1944


In [150]:
gbm.save_model("model.lgb")

In [34]:
test2 = pd.pivot_table(df_test,index=['id'],columns = ['day'], values = 'predict').values

test2[test2<0] = 0


C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in less
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
submission = pd.concat([submission.loc[:,'id'],pd.DataFrame(test2, columns = submission.columns[1:])],axis=1)
submission = round(submission.fillna(0))
submission.to_csv('submission_baseline3_rolling .csv',index=False)

In [55]:
submission


,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HOBBIES_1_002_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,HOBBIES_1_004_CA_1_validation,2.0,2.0,1.0,1.0,2.0,3.0,3.0,2.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
4,HOBBIES_1_005_CA_1_validation,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60976,FOODS_3_824_WI_3_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60977,FOODS_3_825_WI_3_evaluation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60978,FOODS_3_826_WI_3_evaluation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
df_train.shape[0]-snap.sum()

28884852.0

In [32]:
snap = np.zeros(df_train.shape[0])

In [33]:
snap[[0,1,2]]=1
snap

array([1., 1., 1., ..., 0., 0., 0.])